In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import FunctionTransformer


In [2]:
# Improve this next time
# Create a directory for datasets

def get_data(data_link):
    data = pd.read_csv(data_link)
    return data

train_data = get_data('https://raw.githubusercontent.com/wyx-smrf/Windows-Misc/main/Titanic%20Project/train.csv')
test_data = get_data('https://raw.githubusercontent.com/wyx-smrf/Windows-Misc/main/Titanic%20Project/test.csv')

### Pipeline

In [3]:
# Pipeline for categorical columns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))

preprocessing = make_column_transformer((cat_pipeline, make_column_selector(dtype_include=object)), remainder='drop')

In [4]:
process_df = preprocessing.fit_transform(train_data)
process_df

<891x1724 sparse matrix of type '<class 'numpy.float64'>'
	with 4455 stored elements in Compressed Sparse Row format>

### Pandas Pipeline 1: Delete irrelevant columns

In [5]:
params = {'dataframe': train_data, 
          'listed_delete_cols': ['PassengerId', 'Name', 'Ticket', 'Cabin']}

def delete_cols(dataframe, listed_delete_cols):
    new_dataframe = dataframe.drop(columns = listed_delete_cols, axis=1)
    return new_dataframe

removecols_df = delete_cols(**params)
removecols_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


### Pandas Pipeline 2: Add additional columns

In [6]:
column_operation_params = {'dataframe': removecols_df, 
                           'num_col_1': 'SibSp', 
                           'num_col_2': 'Parch',
                           'result_colname': 'Companion'}

def column_operation(dataframe, num_col_1, num_col_2, result_colname):
    dataframe[result_colname] = dataframe[num_col_1] + dataframe[num_col_2]
    return dataframe

addcols_df = column_operation(**column_operation_params)
addcols_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Companion
0,0,3,male,22.0,1,0,7.2500,S,1
1,1,1,female,38.0,1,0,71.2833,C,1
2,1,3,female,26.0,0,0,7.9250,S,0
3,1,1,female,35.0,1,0,53.1000,S,1
4,0,3,male,35.0,0,0,8.0500,S,0
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,0
887,1,1,female,19.0,0,0,30.0000,S,0
888,0,3,female,NaN,1,2,23.4500,S,3
889,1,1,male,26.0,0,0,30.0000,C,0


In [21]:
column_operation_params = {'dataframe': removecols_df, 
                           'num_col_1': 'Fare', 
                           'num_col_2': 'Companion',
                           'result_colname': 'Indiv_Price'}

def column_operation(dataframe, num_col_1, num_col_2, result_colname):
    dataframe[result_colname] = dataframe[num_col_1] / dataframe[num_col_2]
    
    dataframe[result_colname].replace([np.inf, -np.inf], np.nan, inplace=True)
    dataframe[result_colname].fillna(dataframe[num_col_1], inplace=True)
    
    return dataframe

divcols_df = column_operation(**column_operation_params)
divcols_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Companion,Indiv_Price
0,0,3,male,22.0,1,0,7.2500,S,1,7.250000
1,1,1,female,38.0,1,0,71.2833,C,1,71.283300
2,1,3,female,26.0,0,0,7.9250,S,0,7.925000
3,1,1,female,35.0,1,0,53.1000,S,1,53.100000
4,0,3,male,35.0,0,0,8.0500,S,0,8.050000
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,0,13.000000
887,1,1,female,19.0,0,0,30.0000,S,0,30.000000
888,0,3,female,NaN,1,2,23.4500,S,3,7.816667
889,1,1,male,26.0,0,0,30.0000,C,0,30.000000


### Pandas Pipeline 3: Impute Missing Values based on group statistics

In [25]:
# Age Conditional Imputing

conditional_fillna_params = {'dataframe':       divcols_df,
                             'cat_col_1':       'Pclass', 
                             'cat_col_2':       'Sex', 
                             'num_col':         'Age', 
                             'impute_strategy': 'mean'}

def conditional_fillna(dataframe, cat_col_1, cat_col_2, num_col, impute_strategy):
  """
  Impute the missing values in a column using central tendency. 
  The values of the central tendency is obtained by the statistic of the grouped data

  Args:
    dataframe (DataFrame): Two-dimensional data structure stored in a tabular format.
    cat_col_1 (Series): Categorical column to be used in data grouping
    cat_col_2 (Series): Categorical column to be used in data grouping
    num_col (Series): Numerical Column to be used to compute for numeric values among different data groups
    impute_strategy (String): The method on which to impute the missing values

  Returns:
    DataFrame: Imputed missing values based on the statistic from other column/s
  """
  # Create a grouping for the dataset
  grouped_data = dataframe.groupby([cat_col_1, cat_col_2])[num_col]

  # Fill missing values in a column using the defined impute strategy
  dataframe[num_col].fillna(grouped_data.transform(impute_strategy), inplace=True)

  # Optional: Covert the imputed numerical values into an integer
  dataframe[num_col] = dataframe[num_col].round(0).astype(int)

  return dataframe

In [27]:
age_impute_df = conditional_fillna(**conditional_fillna_params)
age_impute_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Companion,Indiv_Price
0,0,3,male,22,1,0,7.2500,S,1,7.250000
1,1,1,female,38,1,0,71.2833,C,1,71.283300
2,1,3,female,26,0,0,7.9250,S,0,7.925000
3,1,1,female,35,1,0,53.1000,S,1,53.100000
4,0,3,male,35,0,0,8.0500,S,0,8.050000
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27,0,0,13.0000,S,0,13.000000
887,1,1,female,19,0,0,30.0000,S,0,30.000000
888,0,3,female,22,1,2,23.4500,S,3,7.816667
889,1,1,male,26,0,0,30.0000,C,0,30.000000


---

# This part above is already polished

In [ ]:
g = sns.FacetGrid(tips, col="time", row="sex")
g.map_dataframe(sns.histplot, x="total_bill", binwidth=2, binrange=(0, 60))

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = ["longitude", "latitude", "housing_median_age", "total_rooms",
               "total_bedrooms", "population", "households", "median_income"]
cat_attribs = ["ocean_proximity"]


make_column_selector(dtype_include=object)


preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

In [ ]:
process_df.loc[:, process_df.columns != 'Age']

In [ ]:
process_df.hist(figsize=(20,20))
plt.show()

In [ ]:
g = sns.FacetGrid(process_df, col="Pclass",  row="Sex")
g.map_dataframe(sns.histplot, x="Age")

In [ ]:
g = sns.FacetGrid(process_df, col="Embarked", row="Sex")
g.map_dataframe(sns.histplot, x="Pclass")
plt.show()

In [ ]:
process_df = conditional_fillna(**params)

In [ ]:
process_df

In [ ]:



df_imputed['Age'] = df_imputed['Age'].fillna(df_imputed.groupby(['Pclass', 'Sex'])['Age'].transform('mean'))

In [ ]:
def monkey_patch_get_signature_names_out():
    """Monkey patch some classes which did not handle get_feature_names_out()
       correctly in Scikit-Learn 1.0.*."""
    from inspect import Signature, signature, Parameter
    import pandas as pd
    from sklearn.impute import SimpleImputer
    from sklearn.pipeline import make_pipeline, Pipeline
    from sklearn.preprocessing import FunctionTransformer, StandardScaler

    default_get_feature_names_out = StandardScaler.get_feature_names_out

    if not hasattr(SimpleImputer, "get_feature_names_out"):
      print("Monkey-patching SimpleImputer.get_feature_names_out()")
      SimpleImputer.get_feature_names_out = default_get_feature_names_out

    if not hasattr(FunctionTransformer, "get_feature_names_out"):
        print("Monkey-patching FunctionTransformer.get_feature_names_out()")
        orig_init = FunctionTransformer.__init__
        orig_sig = signature(orig_init)

        def __init__(*args, feature_names_out=None, **kwargs):
            orig_sig.bind(*args, **kwargs)
            orig_init(*args, **kwargs)
            args[0].feature_names_out = feature_names_out

        __init__.__signature__ = Signature(
            list(signature(orig_init).parameters.values()) + [
                Parameter("feature_names_out", Parameter.KEYWORD_ONLY)])

        def get_feature_names_out(self, names=None):
            if callable(self.feature_names_out):
                return self.feature_names_out(self, names)
            assert self.feature_names_out == "one-to-one"
            return default_get_feature_names_out(self, names)

        FunctionTransformer.__init__ = __init__
        FunctionTransformer.get_feature_names_out = get_feature_names_out

monkey_patch_get_signature_names_out()

In [ ]:
def ratio_name(FunctionTransformer, input_features):
    return ["coltransformation"]  # feature names out

def column_addition(X):
    return X[:, [0]] + X[:, [1]]

def addition_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_addition, feature_names_out=ratio_name),
    ) # Change later


def column_divison(X):
    return X[:, [0]] / X[:, [1]]

def division_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_divison, feature_names_out=ratio_name),
    ) # Change later


preprocessing = ColumnTransformer(
    transformers = [("Companions", addition_pipeline(), ["SibSp", "Parch"]), 
                    ("Price_per_head", division_pipeline(), ["Fare", "Companions"])], 
    remainder='passthrough')

In [ ]:
housing_prepared = preprocessing.fit_transform(removecols_data)
housing_prepared.shape

In [ ]:
preprocessing.get_feature_names_out()

---

In [ ]:
A = [[7,2,3],[4,np.nan,6],[10,5,np.nan]]

In [ ]:
A

In [ ]:
pd.DataFrame(A)